# Download Yellow Taxi Data (2019.12~2020.11)
# It is not recommended to download with the following code (too slow)

In [1]:
import pandas as pd
import numpy as np

In [2]:
psyc = pd.read_csv("Negotiation+Tactics+_+2021_September+6,+2021_17.35.csv")
psyc = psyc.drop('Duration (in seconds)', 1)
pd.set_option("display.max_rows", None, "display.max_columns", None)
psyc = psyc.iloc[:, 7:]

psyc = psyc.drop(['RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude', 'UserLanguage', 'Q_RecaptchaScore', 'Q1', 'Gender_3_TEXT'],  1)
psyc = psyc.drop(['D9 - Parent Topics', 'D9 - Sentiment Polarity', 'D9 - Sentiment Score', 'D9 - Sentiment', 'D9 - Topic Sentiment Label', 'D9 - Topic Sentiment Score', 'D9 - Topics'], 1)

psyc["len_DistributionChannel"] = np.NAN

for i in range(len(psyc['DistributionChannel'])):
    psyc["len_DistributionChannel"][i] = len(psyc['DistributionChannel'][i])

psyc = psyc.loc[(psyc['len_DistributionChannel'] > 7) & (psyc['len_DistributionChannel'] < 20)]
psyc = psyc.drop(['DistributionChannel', 'len_DistributionChannel'], 1)

psyc['Dignity_1'] = psyc['Dignity_1'].astype(float)
psyc['SINS_1'] = psyc['SINS_1'].astype(float)
psyc['Gender'] = psyc['Gender'].astype(float)

psyc = psyc.loc[(psyc['Dignity_1'] > 0) & (psyc["SINS_1"] > 0) & (psyc["Gender"] > 0)]

psyc = psyc[psyc['FBorn'].notna()]
psyc = psyc[psyc['MBorn'].notna()]
psyc = psyc[psyc['Born'].notna()]

psyc = psyc.reset_index()
psyc = psyc.drop(['index'], 1)

psyc["FBorn"] = psyc["FBorn"].str.lower()
psyc["MBorn"] = psyc["MBorn"].str.lower()
psyc["Born"] = psyc["Born"].str.lower()

<ipython-input-2-777feb1b03b2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psyc["len_DistributionChannel"][i] = len(psyc['DistributionChannel'][i])


In [3]:
dignity = ['australia', 'uak', 'estonia', 'french', 'england', 'australua', 'belgium', 'new zealand', 'nz', 'italy', 'qld', 'nsw', 'united states of america', 'iceland', 'arkansas, usa', 'kerang', 'victoria', 'melbourne', 'uk', 'holland', 'scotland', 'aus - perth', 'greece', 'little rock, arkansas, usa', 'united states', 'malta', 'austria', 'morwell', 'london', 'sydney', 'los angeles', 'spain', 'sale', 'shap, uk', 'france', 'western europe', 'united kingdom', 'healesville', ]
face = ['china', 'cn', 'indonesia', 'singapore', 'hong kong', 'korea', 'thailand', 'malaysia', 'vietnam', 'japan', 'cambodia', 'shanghai, china', 'philippines', 'indonesi', 'asia']

psyc["Father"] = np.NaN
psyc["Mother"] = np.NaN
psyc["Me"] = np.NaN

for i in range(len(psyc['FBorn'])):
    for j in dignity:
        if j in psyc['FBorn'][i]:
            psyc["Father"][i] = 1
            break
    for k in face:
        if k in psyc['FBorn'][i]:
            psyc["Father"][i] = 2
            break

for i in range(len(psyc['MBorn'])):
    for j in dignity:
        if j in psyc['MBorn'][i]:
            psyc["Mother"][i] = 1
            break
    for k in face:
        if k in psyc['MBorn'][i]:
            psyc["Mother"][i] = 2
            break

for i in range(len(psyc['Born'])):
    for j in dignity:
        if j in psyc['Born'][i]:
            psyc["Me"][i] = 1
            break
    for k in face:
        if k in psyc['Born'][i]:
            psyc["Me"][i] = 2
            break

<ipython-input-3-e9397b322259>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psyc["Father"][i] = 1
<ipython-input-3-e9397b322259>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psyc["Father"][i] = 2
<ipython-input-3-e9397b322259>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psyc["Mother"][i] = 1
<ipython-input-3-e9397b322259>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See

In [4]:
psyc = psyc[psyc['Mother'].notna()]
psyc = psyc[psyc['Father'].notna()]
psyc = psyc[psyc['Me'].notna()]
psyc = psyc.reset_index()
psyc = psyc.drop(['index'], 1)

psyc["Mother"][4] == psyc["Me"][4]

psyc["Bio_birth"] = np.NaN

for i in range(len(psyc["Me"])):
    all_birth = []
    all_birth.append(psyc["Father"][i])
    all_birth.append(psyc["Mother"][i])
    all_birth.append(psyc["Me"][i])

    birth_unique = list(set(all_birth))

    if len(birth_unique) <= 1:
        if birth_unique[0] == 2:
            psyc["Bio_birth"][i] = 1
        else:
            psyc["Bio_birth"][i] = 0
    else:
        psyc["Bio_birth"][i] = 2


<ipython-input-4-33d43d77accb>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psyc["Bio_birth"][i] = 0
<ipython-input-4-33d43d77accb>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psyc["Bio_birth"][i] = 2
<ipython-input-4-33d43d77accb>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psyc["Bio_birth"][i] = 1


In [5]:
psyc.to_csv('Data with pure face base on FMM Birthplace.csv')
psyc = psyc.drop(['Father', 'Mother', 'Me', 'MBorn', 'FBorn', 'Born'], 1)
psyc.iloc[:, 1:50] = psyc.iloc[:, 1:50].astype(float)

In [6]:
psyc['Dignity_score'] = (psyc["Dignity_1"] + psyc['Dignity_2'] + psyc['Dignity_3'] + psyc['Dignity_4'] + psyc['Dignity_5'] + psyc['Dignity_6']) * (6/15)
psyc['Face_score'] = (psyc['Face_1'] + psyc['Face_2'] + psyc['Face_3'] + psyc['Face_4'] + psyc['Face_5']) * (5/15)
psyc['Honour_score'] = (psyc['Honour_1'] + psyc['Honour_2'] + psyc['Honour_3'] + psyc['Honour_4']) * (4/15)

In [7]:
psyc['Bio_Score'] = np.nan
for i in range(len(psyc['Dignity_score'])):
    if psyc['Dignity_score'][i] >= 10:
        if psyc['Face_score'][i] >= 8:
            if psyc['Honour_score'][i] < 7:
                psyc['Bio_Score'][i] = 2
            else:
                psyc['Bio_Score'][i] = np.NaN
        else:
            if psyc['Honour_score'][i] < 7:
                psyc['Bio_Score'][i] = 0
            else:
                psyc['Bio_Score'][i] = np.NaN
    else:
        if psyc['Face_score'][i] >= 8:
            if psyc['Honour_score'][i] < 7:
                psyc['Bio_Score'][i] = 1
            else:
                psyc['Bio_Score'][i] = np.NaN
        else:
            psyc['Bio_Score'][i] = np.NaN

psyc['Bio_Birth'] = psyc['Bio_birth']
psyc = psyc.drop(['Bio_birth'], 1)
psyc

<ipython-input-7-284a3fc19ec3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psyc['Bio_Score'][i] = 0
<ipython-input-7-284a3fc19ec3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psyc['Bio_Score'][i] = 2
<ipython-input-7-284a3fc19ec3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psyc['Bio_Score'][i] = 1
<ipython-input-7-284a3fc19ec3>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

,ResponseId,Dignity_1,Dignity_2,Dignity_3,Dignity_4,Dignity_5,Dignity_6,Face_1,Face_2,Face_3,Face_4,Face_5,Honour_1,Honour_2,Honour_3,Honour_4,SINS_1,SINS_2,SINS_3,SINS_4,SINS_5,SINS_6,SINS_7,SINS_8,SINS_9,SINS_10,SINS_11,SINS_12,SINS_13,SINS_14,SINS_15,SINS_16,SINS_17,SINS_18,SINS_19,SINS_20,SINS_21,SINS_22,SINS_23,SINS_24,SINS_25,SINS_26,SINS_27,SINS_28,SINS_29,SINS_30,SINS_31,SINS_32,Age,Gender,D6_1,D6_2,D6_3,D7,D7_9_TEXT,D8_1,D8_2,D8_3,D9,D10_1,D10_2,D10_3,Dignity_score,Face_score,Honour_score,Bio_Score,Bio_Birth
0,R_1Qa1Sep4YyfVsk7,7.0,6.0,6.0,6.0,2.0,5.0,5.0,2.0,5.0,3.0,5.0,3.0,6.0,4.0,4.0,1.0,1.0,1.0,4.0,1.0,5.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,5.0,1.0,3.0,5.0,2.0,2.0,2.0,1.0,1.0,3.0,1.0,3.0,2.0,1.0,4.0,22.0,2.0,5,4,4,10,NaN,NaN,NaN,NaN,Italian,4,4,3,12.8,6.666667,4.533333,0.0,0.0
1,R_3pi54AAbuhkIfxs,6.0,5.0,5.0,6.0,6.0,3.0,7.0,7.0,7.0,5.0,6.0,6.0,7.0,6.0,6.0,2.0,2.0,1.0,3.0,2.0,6.0,5.0,1.0,1.0,2.0,4.0,1.0,3.0,4.0,2.0,2.0,5.0,2.0,2.0,4.0,2.0,3.0,3.0,1.0,3.0,4.0,4.0,2.0,4.0,2.0,1.0,6.0,22.0,2.0,5,4,4,10,NaN,NaN,NaN,NaN,Dutch/Australian,4,4,3,12.4,10.666667,6.666667,2.0,0.0
2,R_1mEDzu6CZGZui3j,6.0,4.0,7.0,6.0,7.0,7.0,5.0,7.0,6.0,6.0,7.0,5.0,7.0,5.0,2.0,5.0,3.0,1.0,3.0,3.0,4.0,6.0,6.0,3.0,5.0,4.0,4.0,4.0,4.0,4.0,6.0,5.0,5.0,4.0,6.0,4.0,4.0,5.0,5.0,4.0,6.0,6.0,5.0,5.0,5.0,4.0,5.0,21.0,2.0,5,5,5,10,NaN,NaN,NaN,NaN,Australian,5,5,5,14.8,10.333333,5.066667,2.0,0.0
3,R_2s0DOtaLQmRUnQZ,7.0,6.0,6.0,7.0,7.0,7.0,6.0,5.0,5.0,2.0,6.0,6.0,6.0,6.0,6.0,1.0,3.0,1.0,2.0,1.0,5.0,3.0,2.0,2.0,5.0,5.0,5.0,1.0,5.0,2.0,2.0,5.0,2.0,3.0,6.0,5.0,3.0,1.0,3.0,3.0,5.0,5.0,3.0,3.0,3.0,3.0,5.0,20.0,1.0,4,2,2,9,Singapore,4,4,2,Australian,4,2,1,16.0,8.000000,6.400000,2.0,0.0
4,R_1gGFTvIB7JfQYe4,6.0,6.0,7.0,7.0,7.0,5.0,5.0,4.0,6.0,3.0,5.0,6.0,5.0,5.0,6.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,6.0,3.0,2.0,3.0,5.0,3.0,2.0,1.0,2.0,2.0,2.0,2.0,4.0,1.0,1.0,3.0,34.0,2.0,4,5,5,9,United Kingdom,3,2,5,Chinese,4,5,5,15.2,7.666667,5.866667,0.0,2.0
5,R_2znGLNFVs1F92gz,5.0,6.0,6.0,7.0,7.0,6.0,4.0,5.0,3.0,4.0,4.0,5.0,6.0,5.0,4.0,3.0,3.0,2.0,5.0,3.0,6.0,5.0,3.0,5.0,5.0,4.0,3.0,3.0,6.0,4.0,3.0,6.0,5.0,5.0,6.0,4.0,5.0,4.0,4.0,5.0,6.0,6.0,4.0,6.0,5.0,4.0,6.0,22.0,2.0,5,5,4,10,NaN,NaN,NaN,NaN,Australian,5,5,4,14.8,6.666667,5.333333,0.0,0.0
6,R_1dh4yM2Lpkd4rRC,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,6.0,6.0,6.0,6.0,5.0,6.0,7.0,1.0,2.0,1.0,3.0,2.0,2.0,3.0,2.0,2.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,3.0,5.0,3.0,2.0,2.0,3.0,2.0,2.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,3.0,21.0,2.0,4,4,4,10,NaN,NaN,NaN,NaN,Aussie,4,4,4,12.0,9.333333,6.400000,2.0,0.0
7,R_2c0jsYZPXvs61vs,6.0,5.0,6.0,5.0,5.0,4.0,6.0,5.0,5.0,4.0,5.0,5.0,4.0,5.0,5.0,2.0,1.0,1.0,3.0,1.0,3.0,4.0,1.0,1.0,3.0,3.0,2.0,2.0,2.0,2.0,1.0,4.0,3.0,1.0,4.0,2.0,4.0,1.0,2.0,2.0,2.0,3.0,2.0,4.0,3.0,3.0,3.0,21.0,2.0,4,3,4,10,NaN,NaN,NaN,NaN,Irish,2,1,1,12.4,8.333333,5.066667,2.0,0.0
8,R_1I5TwSJLQOP3B29,6.0,6.0,6.0,6.0,5.0,5.0,5.0,3.0,5.0,3.0,5.0,5.0,6.0,5.0,5.0,2.0,2.0,1.0,5.0,3.0,6.0,5.0,2.0,2.0,3.0,2.0,2.0,2.0,5.0,3.0,3.0,5.0,5.0,3.0,5.0,5.0,3.0,5.0,3.0,3.0,5.0,5.0,3.0,5.0,5.0,3.0,5.0,22.0,2.0,4,5,4,10,NaN,NaN,NaN,NaN,Too broad to specify one - European,1,1,1,13.6,7.000000,5.600000,0.0,0.0
9,R_23TvkEceS0LYXxw,5.0,6.0,5.0,5.0,5.0,3.0,6.0,5.0,6.0,3.0,4.0,3.0,5.0,3.0,3.0,2.0,1.0,1.0,4.0,1.0,6.0,5.0,2.0,1.0,2.0,3.0,2.0,1.0,3.0,1.0,1.0,4.0,5.0,3.0,4.0,4.0,4.0,3.0,4.0,3.0,3.0,5.0,3.0,3.0,3.0,4.0,5.0,21.0,2.0,3,2,2,10,NaN,NaN,NaN,NaN,Italian and Maltese,4,4,4,11.6,8.000000,3.733333,2.0,0.0


In [8]:
psyc.to_csv('Data with pure face base on general score.csv')